In [ ]:
pip install pandas openpyxl lxml


### Read and parse the PLM XML

In [ ]:
import pandas as pd
from lxml import etree
from itertools import islice
from collections import defaultdict

In [ ]:
PLMXML_PATH = 'instance_graf.xml'

In [ ]:


plm_xml_df = pd.read_xml(PLMXML_PATH, parser='lxml')

In [ ]:
def flatten_xml(node, key_prefix=()):
    """
    Walk an XML node, generating tuples of key parts and values.
    """

    # Copy tag content if any
    text = (node.text or '').strip()
    if text:
        yield key_prefix, text

    # Copy attributes
    for attr, value in node.items():
        yield key_prefix + (attr,), value

    # Recurse into children
    for child in node:
        yield from flatten_xml(child, key_prefix + (child.tag,))


def dictify_key_pairs(pairs, key_sep='-'):
    """
    Dictify key pairs from flatten_xml, taking care of duplicate keys.
    """
    out = {}

    # Group by candidate key.
    key_map = defaultdict(list)
    for key_parts, value in pairs:
        key_map[key_sep.join(key_parts)].append(value)

    # Figure out the final dict with suffixes if required.
    for key, values in key_map.items():
        if len(values) == 1:  # No need to suffix keys.
            out[key] = values[0]
        else:  # More than one value for this key.
            for suffix, value in enumerate(values, 1):
                out[f'{key}{key_sep}{suffix}'] = value

    return out

In [ ]:
path = 'digi_150_VCC110_V536_2021-08-18.plmxml'
parser = etree.XMLParser(encoding='utf-8', recover=True)
tree = etree.parse(path, parser)

In [ ]:
rows = [dictify_key_pairs(flatten_xml(row)) for row in tree.iter()]

In [ ]:
len(rows)

In [ ]:
selected_rows_iter =islice(rows, 1500000, None)
for row in selected_rows_iter:
    print(row)

In [ ]:
iterable = tree.iter()
print(f"TYPE {type(iterable)}" )
